### Электрон в одномерной полубесконечной потенциальной яме

In [ ]:
using Plots, Roots

In [ ]:
plot([2,2], [2,4], line = (:arrow, 3, :black), annotations=(2.02, 4, text("U(x)", :left)) )
plot!([1.75,3], [2,2], line = (:arrow, 1, :black), annotations=(3, 1.92, text("x", :down)))
plot!([2,2.4,2.4,3], [2,2,3,3], line = ( 3, :black), leg=false, ticks=nothing, border=:none )

annotate!([(2.4, 3.05, text("U0", :right)), (2.4, 1.92, text("a", :center)), 
        (2, 1.92, text("0", :center)), (2.2, 2.5, text("I")), (2.6, 2.5, text("II")) ])

Стационарное уравнение Шрёдингера $H\psi=E\psi$ для данной системы удобно разбить на выражения для областей I и II (Помним о линейности и суперпозиции):

$$
H_i\psi_i=E\psi_i
$$
$i=1,2$, $H_1 = T+0$, $H_2 = T + U_0$
$$
\begin{cases}
\frac{d^2\psi_1}{dx^2} + \frac{2m}{\hbar^2}E\psi_1 = 0
\\
\frac{d^2\psi_2}{dx^2} - \frac{2m}{\hbar^2}(U_0-E)\psi_2 = 0
\end{cases}
$$

$$
\begin{cases}
k_1 = \sqrt{ \frac{2mE}{\hbar^2} }
\\
k_2 = \sqrt{ \frac{2m(U_0-E)}{\hbar^2} }
\end{cases}
$$

$$
\begin{cases}
\psi_1(x) = A\sin(k_1x + \alpha)
\\
\psi_2(x) = Be^{k_2x} + Ce^{-k_2x}
\end{cases}
$$

По условию $\psi_1(0) = 0\, \rightarrow\, \alpha=0$, $\psi_2(\infty) = 0\, \rightarrow\, B=0$. Далее нужно найти допустимые значения Е, для этого используем условия сшивки (непрерывность ВФ и их производных в точке х=а):

$$
\begin{cases}
A\sin(k_1a) = Ce^{-k_2a}
\\
k_1A\cos(k_1a) = -k_2Ce^{-k_2a}
\end{cases}
$$

Не вдаваясь в анализ можно сразу кинуться решать эту систему относительно Е

$$
det\begin{vmatrix}
\sin(k_1a) & e^{-k_2a}\\ 
k_1\cos(k_1a) & -k_2e^{-k_2a}
\end{vmatrix} = 0
$$

Вы ведь хорошо разбираетесь в численых методах решения трансцендентных уравнений? Если нет, то идём дальше. Делим первое уравнение на второе, и стараемся прийти к чему-то проще тангенса, что можно было бы решить графически:

$$
\frac 1 {k_1}tg k_1a = -\frac 1 {k_2}\, \rightarrow\, ctg^2k_1a = \frac{k_2^2}{k_1^2} = \frac{U_0-E}E = \frac{U_0}E-1\, \rightarrow\, \frac 1{\sin^2k_1a}-1 = \frac{U_0}E-1\, \rightarrow
$$
$$
\rightarrow\, \sin^2k_1a = \frac{E}{U_0} = \frac{2mE}{\hbar^2}a^2\frac{\hbar^2}{2ma^2}\frac{1}{U_0} = k_1^2a^2\frac{\hbar^2}{2ma^2U_0}
$$

$$
\sin k_1a = \pm\sqrt{\frac{\hbar^2}{2ma^2U_0}}k_1a
$$

Это уравнение можно решить графически относительно $k_1a$. Используя условие $\tan k_1a<0$, получаем ограничение на аргумент $\frac \pi 2+\pi n<k_1a<\pi+\pi n,\, n\in (0,\infty)$. Также пригодится условие существования хотябы одного энергетического уровня: $U_0a^2>\frac{\pi^2\hbar^2}{8m}$. Пересечения будут происходить до $k_1a = \sqrt{\frac{2mU_0a^2}{\hbar^2}}=\gamma$. A находим из условий нормировки, С - из условий на границе.

In [ ]:
#const ħ = 6.5821e-16 # eV*s
const ħ = 1.0546e-34 # J*s
const m = 9.1094e-31 # kg
const q = 1.6022e-19 # Kl

k(E) = sqrt(2m*E*q)/ħ # 1/m

In [ ]:
function nrg(U0, a = 10e-10)
    
    γ = k(U0)*a # безразмерная
    ka = [0:0.005γ:γ;]
    # ищем точки пересечения
    solup = fzeros(x->sin(x)-x/γ, 0, γ) # up
    soldn = fzeros(x->sin(x)+x/γ, 0, γ) # down
    solns = sort( [solup; soldn] )
    # отсеиваем неудовлетворяющие ограничениям:
    function conds(x)
        N = γ/π
        ret = false
        for n = 0:N
            ret |= (0.5π+π*n < x < π+π*n) # хитро?
        end
        return ret
    end
    En = filter(conds, solns)
    
    plot(sin, ka, line = (3) )
    plot!(x->  x/γ, ka, line = (3) )
    plot!(x-> -x/γ, ka, line = (3) )
    scatter!( sin, solns, marker=(20, 0.2, :orange) )
    scatter!( sin, En, marker=(10, 0.8, :red), legend = false )
    title!("Allowed energy values")
    
    return ħ^2  / (2m*q*a^2) * En.^2
end

In [ ]:
E = nrg(5, 20e-10) # 5 эВ, 20 Ангстрем
xaxis!("ka")

In [ ]:
# красные - которые подходят по условиям
E # разрешенные уровни энергии

In [ ]:
function ψ(n = 1, U0 = 1, a = 10e-10) # номер энерг-го ур-ня, ширина ямы
    
    x1 = [0:0.01a:a;]
    x2 = [a:0.01a:1.5a;]
    k1 = k( E[n] ) # 1/m
    k2 = k( U0-E[n] )
    
    A = sqrt( 4k1/(2k1*a - sin(2k1*a)) )
    #C = sqrt(2k2) * exp(a*k2) # ??? из нормировки
    C = A*sin(k1*a)/exp(-k2*a)
    
    ψ1(x) = A*sin( k1*x )
    ψ2(x) = C*exp(-k2*x )
    
    plot(ψ1, x1, line = (3), legend = false )
    plot!(ψ2, x2, line = (3), xaxis = ("x, A") )
    vline!([a])
    title!("E = $(round(E[n], digits = 3)) eV")
end

In [ ]:
ψ(3, 5, 20e-10) # условия ставить те же

### Электрон в одномерной потенциальной яме конечной глубины

In [ ]:
plot([0,0], [-1,4], line = (:arrow, 1, :black), annotations=(0.2, 4, text("U(x)", :left)) )
plot!([-4,4], [0,0], line = (:arrow, 1, :black), annotations=(4, 0.2, text("x", :up)))
plot!([-4,-2,-2,2,2,4 ], [3,3,0,0,3,3 ], line = ( 3, :black), leg=false, ticks=nothing, border=:none )

annotate!([(2, 3.2, text("U0", :up)), (2, -0.2, text("a/2", :center)), (-2, -0.2, text("-a/2", :center)) ])

Как и раньше, расписываем стационарное уравнение Шрёдингера $H\psi=E\psi$ для каждой области:

$$
H_i\psi_i=E\psi_i
$$
$i=1, 2, 3$, $H_2 = T+0$, $H_{1,3} = T + U_0$
$$
\begin{cases}
\frac{d^2\psi_1}{dx^2} - \frac{2m}{\hbar^2}(U_0-E)\psi_1 = 0
\\
\frac{d^2\psi_2}{dx^2} + \frac{2m}{\hbar^2}E\psi_2 = 0
\\
\frac{d^2\psi_3}{dx^2} - \frac{2m}{\hbar^2}(U_0-E)\psi_3 = 0
\end{cases}
$$

$$
\begin{cases}
\gamma = \sqrt{ \frac{2mE}{\hbar^2} }
\\
\beta = \sqrt{ \frac{2m(U_0-E)}{\hbar^2} }
\end{cases}
$$

$$
\begin{cases}
\psi_1(x) = A_1e^{\beta x}
\\
\psi_2(x) = A_2e^{i\gamma x} + B_2e^{-i\gamma x}
\\
\psi_3(x) = B_3e^{-\beta x}
\end{cases}
$$

$B_1 = A_3 = 0$ из условия ограничения на бесконечности.

Используем условие непрерывности волновой функции и её производной на границах потенциальной ямы:

$$
\begin{cases}
\psi_1(-\frac a 2) = \psi_2(-\frac a 2)
\\
\frac 1 {m_b}\frac{\partial\psi_1}{\partial x}\left(-\frac a 2\right) = \frac 1 {m_a}\frac{\partial\psi_2}{\partial x}\left(-\frac a 2\right)
\\
\psi_2(\frac a 2) = \psi_3(\frac a 2)
\\
\frac 1 {m_a}\frac{\partial\psi_2}{\partial x}\left(\frac a 2\right) = \frac 1 {m_b}\frac{\partial\psi_3}{\partial x}\left(\frac a 2\right)
\end{cases}
$$

$m_a$ и $m_b$ значения [эффективной массы](https://ru.wikipedia.org/wiki/Эффективная_масса) электрона в веществе А и В.

$$
\begin{cases}
A_1 e^{-\frac a 2 \beta} = A_2 e^{-i\frac a 2 \gamma} + B_2 e^{i\frac a 2 \gamma}
\\
A_1 \frac \beta {m_b} e^{-\frac a 2 \beta} = A_2 \frac{i\gamma}{m_a} e^{-i\frac a 2 \gamma} - B_2 \frac{i\gamma}{m_a} e^{i\frac a 2 \gamma}
\\
A_2 e^{i\frac a 2 \gamma} + B_2 e^{-i\frac a 2 \gamma} = B_3 e^{-\frac a 2 \beta}
\\
A_2 \frac{i\gamma}{m_a} e^{i\frac a 2 \gamma} - B_2 \frac{i\gamma}{m_a} e^{-i\frac a 2 \gamma} = -B_3 \frac \beta {m_b} e^{-\frac a 2 \beta}
\end{cases}
$$

Система имеет нетривиальные решения при равенстве нулю детерминанта:

$$
\begin{vmatrix}
e^{-\frac a 2 \beta} & -e^{-i\frac a 2 \gamma} & -e^{i\frac a 2 \gamma} & 0\\ 
\frac \beta {m_b} e^{-\frac a 2 \beta} & -\frac{i\gamma}{m_a} e^{-i\frac a 2 \gamma} & \frac{i\gamma}{m_a} e^{i\frac a 2 \gamma} & 0\\ 
0 & e^{i\frac a 2 \gamma} & e^{-i\frac a 2 \gamma} & -e^{-\frac a 2 \beta}\\ 
0 & \frac{i\gamma}{m_a} e^{i\frac a 2 \gamma} & -\frac{i\gamma}{m_a} e^{-i\frac a 2 \gamma} & \frac \beta {m_b} e^{-\frac a 2 \beta}
\end{vmatrix} = 0
$$

In [ ]:
using Reduce, Latexify

In [ ]:
using ForceImport
@force using Reduce.Algebra

In [ ]:
function ltx(formula)
    l = latexify(formula)
    res = replace(l, r"\\euler"=>"e")
    latexstring(res)
end

In [ ]:
a1 = :( exp(β*x) )
a2 = :( exp(i*γ*x) )
b2 = :( exp(-i*γ*x) )
b3 = :( exp(-β*x) )

da1 = df(a1, :x)/:m_b
da2 = df(a2, :x)/:m_a
db2 = df(b2, :x)/:m_a
db3 = df(b3, :x)/:m_b

In [ ]:
M = [
    a1 -a2 -b2 0
    da1 -da2 -db2 0
    0 a2 b2 -b3
    0 da2 db2 -db3
]
ltx(M)

In [ ]:
trnsf(s, m) = Algebra.sub((:(x= $s*a/2),:(y=y)),m)

M2 = [ i==1||i==2 ? trnsf(-1, M[i,j]) : trnsf(1, M[i,j]) for i=1:4, j=1:4 ]

In [ ]:
ltx(M2)

In [ ]:
M2 |> det |> ltx

In [ ]:
dM2 = det(M2)

In [ ]:
Expr(:function,:(fun(γ, β, a, m_a, m_b)), dM2 ) |> eval

In [ ]:
# соберется такая функция
fun(γ, β, a, m_a, m_b) = (((m_a ^ 2 * β ^ 2 - m_b ^ 2 * γ ^ 2) - 2 * im * m_a * m_b * β * γ) - 
    ℯ ^ (2 * a * im * γ) * ((m_a ^ 2 * β ^ 2 - m_b ^ 2 * γ ^ 2) + 2 * im * m_a * m_b * β * γ)) / 
        (ℯ ^ (a * im * γ + a * β) * m_a ^ 2 * m_b ^ 2)

In [ ]:
function detE(E; U0 = 1, a = 10e-10)
        k1 = k( E )
        k2 = k( U0-E )
        fun(k1, k2, a, m, m) |> abs
    end

Решения находим графически и уточняем с помощью `fzero`

In [ ]:
plot(detE, [0:0.001:0.75], yaxis=("Y", (0, 1e78) ) )

In [ ]:
fzero(detE, 0.2 ) # xn = 0.19076547740519453

Рассмотрим иной путь решения. Система ВФ может быть представлена в виде:

$$
\begin{cases}
\psi_1(x) = Ce^{\beta x}
\\
\psi_2(x) = A_2e^{i\gamma x} + B_2e^{-i\gamma x} = A\cos(i\gamma x)
\\
\psi_3(x) = Be^{-\beta x}
\end{cases}
$$

Из условий симметрии задачу можно разбить на две части ($\pm\frac a 2$). Для четных решений синус, иначе косинус (~~или наоборот?~~)

$$
\begin{cases}
A \cos{\frac a 2 \gamma} = B e^{-i\frac a 2 \beta}
\\
-A\gamma \sin{\frac a 2 \gamma} = -B\beta e^{-i\frac a 2 \beta}
\end{cases}
$$

$$
\begin{cases}
A \sin{\frac a 2 \gamma} = B e^{-i\frac a 2 \beta}
\\
A\gamma \cos{\frac a 2 \gamma} = -B\beta e^{-i\frac a 2 \beta}
\end{cases}
$$

$$
\gamma \tan{\frac a 2 \gamma} = \beta 
$$
$$
\gamma \cot{\frac a 2 \gamma} = -\beta 
$$
$$
\sqrt E \tan\left( \frac{a\sqrt{2mE}}{2\hbar} \right) = \sqrt{U_0-E}\\
\sqrt E \cot\left( \frac{a\sqrt{2mE}}{2\hbar} \right) = -\sqrt{U_0-E}
$$

In [ ]:
function nrg3b(a = 10e-10, U0 = 1)
    
    e = [0:0.01:U0;]
    E1(E) = tan(0.5a*k(E))
    Sq1(E) = sqrt(U0/E - 1.)
    
    E2(E) = cot(0.5a*k(E))
    Sq2(E) = -sqrt(U0/E - 1.)
    
    plot(E1, e)
    p1 = plot!(Sq1, e)
    plot(E2, e)
    p2 = plot!(Sq2, e)
    plot(p1, p2)
end

In [ ]:
nrg3b()

In [ ]:
fzero(E->tan(5e-10*k(E)) - sqrt(1. /E - 1.), 0.25 ),
fzero(E->cot(5e-10*k(E)) + sqrt(1. /E - 1.), 0.75 )

In [ ]:
function Ψ(En, a = 10e-10, U0 = 1)
    
    k1 = k(En)
    k2 = k(U0-En)
    A1 = 1. # sqrt(k2)*exp(-0.25a*k2)
    A2 = 0.5A1*(1. - im*k2/k1)*exp( 0.5a*(im*k1-k2) )
    B2 = 0.5A1*(1. + im*k2/k1)*exp(-0.5a*(im*k1+k2) )
    B3 = 0.5A1*( (1. - im*k2/k1)*exp( a*im*k1 ) + (1. + im*k2/k1)*exp( -a*im*k1 ) )
    
    x1 = [-a:0.01a:-0.5a;]
    x2 = [-0.5a:0.01a:0.5a;]
    x3 = [0.5a:0.01a:a;]
    
    ψ1(x) = A1*exp(k2*x) |> abs2
    ψ2(x) = A2*exp(im*k1*x) + B2*exp(-im*k1*x) |> abs2
    ψ3(x) = B3*exp(-k2*x) |> abs2
    
    plot(ψ1, x1, line = (3) )
    plot!(ψ2, x2, line = (3) )
    plot!(ψ3, x3, line = (3), xaxis = ("x, A") )
    vline!([-0.5a, 0.5a], legend = false)
    title!("Location probability")
end

In [ ]:
Ψ(0.190765)

In [ ]:
Ψ(0.70278)

То есть при определенных значениях энергии частица не будет находится в центре ямы из-за интерференции ВФ самой с собой. Жуть...